# Import Modules

In [1]:
import torch
import torch.nn as nn
import numpy as np
import math

In [2]:
with open("input.txt", "r", encoding="utf-8") as f:
    lines = f.read()

print("Length of text:", len(lines))
print(f"First 10,000 characters {lines[:10000]}")

Length of text: 1115394
First 10,000 characters First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this i

# Mapping from characters to integers

In [3]:
chars = sorted(list(set(lines)))
vocab_size = len(chars)
print(f"Vocab Size is {vocab_size}")
print(f"First 10 characters in text: {lines[:10]}")
print(f"First 10 characters in vocab: {chars[:15]}")

Vocab Size is 65
First 10 characters in text: First Citi
First 10 characters in vocab: ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B']


## Example of encoding then decoding

In [4]:
vec = {x: y for y, x in enumerate(sorted(set(lines)))}
cev = {y: x for y, x in enumerate(sorted(set(lines)))}
encode = lambda s: [vec[c] for c in s] 
decode = lambda l: ''.join([cev[i] for i in l])

print(encode("I love you"))
print(decode(encode("I love you"))) 

[21, 1, 50, 53, 60, 43, 1, 63, 53, 59]
I love you


# Use PyTorch now

In [5]:
data = torch.tensor(encode(lines), dtype=torch.long)
print(f"Data shape: {data.shape} and Data type: {data.dtype}")
print(f"First 100 characters in data: {data[:100]}")

Data shape: torch.Size([1115394]) and Data type: torch.int64
First 100 characters in data: tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


# Concepts and Ideas 

In [6]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]
print(f"Training data shape: {train_data.shape} and Training data type: {train_data.dtype}")
print(f"Validation data shape: {val_data.shape} and Validation data type: {val_data.dtype}")

Training data shape: torch.Size([1003854]) and Training data type: torch.int64
Validation data shape: torch.Size([111540]) and Validation data type: torch.int64


In [7]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [8]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for i in range(block_size):
    print(f"Input: {x[:i+1]} and Target: {y[i]}")

Input: tensor([18]) and Target: 47
Input: tensor([18, 47]) and Target: 56
Input: tensor([18, 47, 56]) and Target: 57
Input: tensor([18, 47, 56, 57]) and Target: 58
Input: tensor([18, 47, 56, 57, 58]) and Target: 1
Input: tensor([18, 47, 56, 57, 58,  1]) and Target: 15
Input: tensor([18, 47, 56, 57, 58,  1, 15]) and Target: 47
Input: tensor([18, 47, 56, 57, 58,  1, 15, 47]) and Target: 58


In [12]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y
xb, yb = get_batch('train')
print(f"Input shape: {xb.shape} and Input type: {xb.dtype}")
print(f"Target shape: {yb.shape} and Target type: {yb.dtype}")
print(f"Input: {xb} and Target: {yb}")

for b in range(batch_size):
    for i in range(block_size):
        context = xb[b, :i+1]
        target = yb[b, i]
        print(f"Input: {context} and Target: {target}")

Input shape: torch.Size([4, 8]) and Input type: torch.int64
Target shape: torch.Size([4, 8]) and Target type: torch.int64
Input: tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]]) and Target: tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
Input: tensor([24]) and Target: 43
Input: tensor([24, 43]) and Target: 58
Input: tensor([24, 43, 58]) and Target: 5
Input: tensor([24, 43, 58,  5]) and Target: 57
Input: tensor([24, 43, 58,  5, 57]) and Target: 1
Input: tensor([24, 43, 58,  5, 57,  1]) and Target: 46
Input: tensor([24, 43, 58,  5, 57,  1, 46]) and Target: 43
Input: tensor([24, 43, 58,  5, 57,  1, 46, 43]) and Target: 39
Input: tensor([44]) and Target: 53
Input: tensor([44, 53]) and Target: 56
Input: tensor([44, 53, 56]) and Target: 1
Input: tensor([